## Goal, assumptions and description of the approach

Goal of the tool is to suggests number of bikes to be supplied to a given station in the morning to minimise the imbalance between supply and demand of bikes

The challenging aspect of the task is that there is no way of knowing the "true" demand for bikes - we only know how many bikes were used in given time but this is limited by the actual supply of bikes.
While it is not possible to know the true demand my proposition is to estimate the imbalance using the difference of prediction created by two models:
one using "global", seasonal features and other using "local" features (numbers of rentals in a few rolling windows within last days).

If there is a huge disproportion between prediction based on local parameters and more global ones it might mean, that some demand for bikes have not been served. 

Of course to prove or disprove this statement one would need to conduct an experiment on the ground.

Different approach would be to use the number of available bikes as a measure of supply but is very hard to accurately predict on hourly level (and on the other hand: there is no obvious way to aggregate number of available bikes to a daily dataset). This approach could be of course examined if any serious drawbacks of current approach were found.

Also worth noting is that current approach uses daily predictions which would be easier to use by the teams supplying the bikes to the stations.

In the current implementation the "**global**" model is a set of decision trees trained individually for each station. The choice od a decision tree is dictated by non-linear character of relation between rent count and calendar features.

As this is a PoC a few ways of further development can be thought of:

- Clustering stations according to their usage patterns and train more sophisticated models (i.e. XGB regressor or an LSTM) on per-cluster basis. Currently using such models per station or using station number as a one-hot encoded feature would demand either way more time or computing power

- Using additional features, one obvious example that comes to mind is a weather forecast

"**Local**" model in current implementation is a Linear Regression computed per station using number of rentals from last day as a whole, same hour 24 hours ago and same day of week a week ago. Choice of linear regression is dictated by the belief that relation between neighbouring rent counts should be linear.

Final recomendation is based on the difference between prediction of local and global model. 

Note: functions used in the process are documented separately (in docstrings) and are not described thoroughly in this document


In [33]:
import numpy as np
import pandas as pd 
from sklearn.metrics import explained_variance_score, mean_absolute_error
import veturilo_helper as vh
import veturilo_timeseries_function as vtf
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression


In [3]:
hourly_rentals = vh.get_hourly_rentals_df()
hourly_rentals = hourly_rentals[hourly_rentals["uid"] != -1]
hourly_rentals["D"] = hourly_rentals["dt"].dt.to_period("D")

hourly_rentals = vtf.extract_features(hourly_rentals, rolsum_column="rent_count")

/home/podles/projects/veturilo/veturilo_timeseries_function.py:82: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df.loc[:, "weeknum"] = df.loc[:, "dt"].dt.week


## Spliting dataset into train and test

Models are trained on the period 2018-03-01 - 2019-11-30 and tested on the period 2020-06-01 - 2020-11-31

It's worth noting that year 2020 had it's particularities (Veturilo system was shut down for whole month of April and the COVID-19 pandemic might have not yet known implications to users' behavior

Due to limited timeframe experiments with more sophisticated crossvalidation techniques were not conducted

In [4]:
train = hourly_rentals[hourly_rentals["dt"] < "2020-01-01"]
test = hourly_rentals[
    (hourly_rentals["dt"] > "2020-06-15") & (hourly_rentals["dt"] < "2021-01-01")
]


In [5]:
# solving the cold start problem is left outside the scope of this PoC
# but scale of the problem is not that big (it affects only week of the history of each station)
# for now the data containing NaNs in predictions are removed
train = train.dropna()
test = test.dropna()


## Creating the models

Features used in global model, trained on seasonalities are:

- ISO number of week within a year
- Day of the week
- Hour of the day

Features used in local models are:

- Number of rentals within 24 hours but with 24 hours offset (so the *youngest* datapoint used is 24hours old
- Number of rentals within 24 hour a week before
- Number of rentals within 1 hour a day before

The features are selected in a way that enables action with reasonable lead time to supply bikes in the morning

(Almost) no hyperparameter tuning have been performed, as this is only a PoC. Only quick-win used is changing criterion for DT to absolute error, as it fit's exactly our purpose (and indeed yields better result than MSE)

In [10]:
features_list_DT = ["weeknum", "dayofweek", "hour"]
features_list_lin = ["rent_count_48_24", "rent_count_25_24", "rent_count_168_144"]


DT_model_directory = {
    u: vtf.create_model(
        train[train["uid"] == u],
        tgt="rent_count",
        features=features_list_DT,
        mdl_fun=DecisionTreeRegressor,
        max_depth=4,
        criterion="mae" 
    )
    for u in train["uid"].unique()
}

lin_model_directory = {
    u: vtf.create_model(
        train[train["uid"] == u],
        tgt="rent_count",
        features=features_list_lin,
        mdl_fun=LinearRegression,
    )
    for u in train["uid"].unique()
}

params = {
    "global_prediction": {
        "model_directory": DT_model_directory,
        "features_list": features_list_DT,
    },
    "local_prediction": {
        "model_directory": lin_model_directory,
        "features_list": features_list_lin,
    },
}
train = vtf.add_predictions(train, params)
test = vtf.add_predictions(test, params)

daily_train = vtf.aggregate_daily_predictions(train)
daily_test = vtf.aggregate_daily_predictions(test)

## Validating quality of prediction

Quality of hourly prediction of both models is very weak but as our goal is to act using the aggregated, daily imbalances th import fact is that aggregated daily predictions are decent enough for the PoC

Aggregated predictions don't suffer much from overfitting

In [20]:
def prediction_quality_params(df,fun=explained_variance_score):
    return {
        "Local prediction" : fun(df["rent_count"], df["local_prediction"]),
        "Global prediction" : fun(df["rent_count"], df["global_prediction"])
        }

def show_prediction_quality(df_test,df_train,fun=explained_variance_score):
    train_prediction_quality = prediction_quality_params(df_train,fun)
    test_prediction_quality = prediction_quality_params(df_test,fun)

    print("explained variance for test set: ")
    print(f"Local prediction {test_prediction_quality['Local prediction']:.2f}")
    print(f"Global prediction {test_prediction_quality['Global prediction']:.2f}")

    print("explained variance for train set: ")
    print(f"Local prediction {train_prediction_quality['Local prediction']:.2f}")
    print(f"Global prediction {train_prediction_quality['Global prediction']:.2f}")



### Prediction quality for daily aggregates:

In [21]:
show_prediction_quality(daily_test,daily_train)

explained variance for test set: 
Local prediction 0.72
Global prediction 0.57
explained variance for train set: 
Local prediction 0.74
Global prediction 0.59


### Prediction quality for hourly datasets:

As it was mentioned before - quality of hourly prediction is poor and should not be used "as is" (only after aggregation)

In [19]:
show_prediction_quality(test,train)

explained variance for test set: 
Local prediction 0.21
Global prediction 0.23
explained variance for train set: 
Local prediction 0.28
Global prediction 0.31


## Final bike supply recommender

According to our assumption final recommendation of number of bikes supplied to given station on a given day should be a function of difference between predictions of local and global model.

First naive form of this recomender is filtering based on two threshold:

- Absolute number of bikes "missing" (a proxy for which is the difference between predictions)
- Relative threshold: the difference of prediction divided by the global prediction

Exact values of this parameters would depend of technical (and economic) feasibility of bikes supply.
Some order of magnitude can be guessed using average daily rental counts.


In [52]:
def recommender(df,count_threshold,pct_threshold):
    df["pct"] = np.ceil(df["unmet_demand"])/np.ceil(df["global_prediction"])
    return df[(df["unmet_demand"] > count_threshold) & (df["pct"] > pct_threshold)]


def add_recomendation(df,count_threshold,pct_threshold):
    recomendation_df = recommender(df,count_threshold,pct_threshold)
    recomendation_df = recomendation_df[["uid","D"]]
    recomendation_df["RECOMMENDATION"] = 1
    df = df.merge(recomendation_df,how="left",on=["uid","D"])
    df["RECOMMENDATION"] = df["RECOMMENDATION"].fillna(0)
    return df


def plot_prediction(df, uid, dt, cols=["rent_count", "pred"]):
    condition = (df["uid"] == uid) & (df["D"] == dt)
    tmp_df = df[condition]
    tmp_df = tmp_df.set_index("dt")
    tmp_df[cols].plot.line()

In [77]:
with_recommendation = add_recomendation(daily_test,25,0.3)

## Final notes

What is missing from above reasoning is a way of validating the recomendations.

Obvious way as it was stated before would be for instant an experiment where recomendations are applied and number of rented bikes is measured. 

During the experimentation phase number of bikes added to the station could be signifficantly higher than recommendation to check whether there is some more demand not predicted by the tool.